In [2]:
import os 
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns 
sns.set(rc={'figure.figsize': (8,6)})
%matplotlib inline

#SKlearn lib:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 

import torch

from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import Module
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import ReLU
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
from torch.optim import SGD, Adam

from sklearn.metrics import mean_squared_error

In [54]:
main = 'C:\\Users\\joaopaulo\\Desktop'
# main = 'C:\\Users\\joaopaulo\\Desktop\\Gla_data\\PP EPC FULL'
gla_file = os.path.join(main, 'final_merge4.csv')
gla_data = pd.read_csv(gla_file)

print(gla_data.shape) #(331654, 29)
gla_data.head()

(804600, 37)


,index,county,district,oa11,lsoa11,msoa11,postcode,ADDRESS,lat,long,...,Fare_Zone,Deprivation_Index,driving_distance_miles,Median(£)-2011,connec_air_public,connec_air_car,connec_road_car,Gross Domestic Product: Quarter on Quarter growth: CVM SA %,"Construction, cont Q on Q yr ago",Real Estate Activities (period on period growth) %:CVM
0,1787073,GREATER LONDON,LEWISHAM,E00016638,E01003296,E02000664,SE13 5FP,"014 Brick Kiln One, Station Road",51.465100,-0.012095,...,3.0,5.850052,5.81,190000.0,0.438756,0.253657,0.434362,0.3,-0.13,0.5
1,1811124,GREATER LONDON,LEWISHAM,E00016638,E01003296,E02000664,SE13 5FL,"036 River Mill One, Station Road",51.465120,-0.012728,...,3.0,5.850052,5.81,190000.0,0.438756,0.253657,0.434362,0.7,-0.07,-0.4
2,1870838,GREATER LONDON,MERTON,E00017413,E01003447,E02000704,SW20 0AN,"1, Aboyne Drive",51.408534,-0.243936,...,4.0,3.240188,13.92,210000.0,0.360948,0.284701,0.447893,0.5,0.16,1.5
3,359864,GREATER LONDON,BROMLEY,E00003311,E01000683,E02000161,BR6 9UD,"1, Ambrose Close",51.367211,0.089848,...,6.0,-2.300779,17.67,230000.0,0.266449,0.254481,0.464440,0.6,0.23,0.3
4,2144715,GREATER LONDON,RICHMOND UPON THAMES,E00019520,E01003883,E02000797,TW2 6RR,"1, Barneby Close",51.446627,-0.346002,...,5.0,-2.432610,16.01,374950.0,0.392929,0.357184,0.524401,1.2,-0.03,1.5


In [55]:
gla_data.columns

Index(['index', 'county', 'district', 'oa11', 'lsoa11', 'msoa11', 'postcode',
       'ADDRESS', 'lat', 'long', 'property_type', 'new_build', 'duration',
       'category', 'CURRENT_ENERGY_RATING', 'TOTAL_FLOOR_AREA',
       'NUMBER_HABITABLE_ROOMS', 'NUMBER_HEATED_ROOMS',
       'MULTI_GLAZE_PROPORTION', 'WINDOWS_DESCRIPTION', 'PROPERTY_TYPE',
       'BUILT_FORM', 'LODGEMENT_DATE', 'deed_date', 'year', 'quarter',
       'price_paid', 'Fare_Zone', 'Deprivation_Index',
       'driving_distance_miles', 'Median(£)-2011', 'connec_air_public',
       'connec_air_car', 'connec_road_car',
       'Gross Domestic Product: Quarter on Quarter growth: CVM SA %',
       'Construction, cont Q on Q yr ago',
       'Real Estate Activities (period on period growth) %:CVM'],
      dtype='object')

In [57]:
gla_data.dropna(subset=['Fare_Zone', 'Deprivation_Index', 'driving_distance_miles', 'Median(£)-2011', 'NUMBER_HABITABLE_ROOMS', 
                       'NUMBER_HEATED_ROOMS'], inplace=True)
print(gla_data.shape)
##(331654, 29) -> 331643, 29)

(683505, 37)


In [59]:
#Limiting the data:

gla_data2 = gla_data[['property_type', 'new_build', 'duration', 'category',     
       'CURRENT_ENERGY_RATING', 'TOTAL_FLOOR_AREA', 'NUMBER_HABITABLE_ROOMS',
       'NUMBER_HEATED_ROOMS', 'MULTI_GLAZE_PROPORTION', 'WINDOWS_DESCRIPTION',
       'PROPERTY_TYPE', 'BUILT_FORM', 'price_paid', 'Fare_Zone',
       'Deprivation_Index', 'driving_distance_miles', 'Median(£)-2011']]

In [60]:
#Investigate elements within each feature:

for column in gla_data2:
    unique_vals = np.unique(gla_data2[column])
    nr_vals = len(unique_vals)
    if nr_vals <60:
        print('The number of unique values for feature {}: {} - {}'.format(column, nr_vals, unique_vals))
    else:
        print('The number of unique values for feature {}: {}'.format(column, nr_vals))

The number of unique values for feature property_type: 4 - [1. 3. 4. 5.]
The number of unique values for feature new_build: 2 - [0. 1.]
The number of unique values for feature duration: 2 - [0. 1.]
The number of unique values for feature category: 1 - [1.]
The number of unique values for feature CURRENT_ENERGY_RATING: 7 - [1. 2. 3. 4. 5. 6. 7.]
The number of unique values for feature TOTAL_FLOOR_AREA: 24557
The number of unique values for feature NUMBER_HABITABLE_ROOMS: 46 - [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 25. 30. 31. 33. 34. 36. 40. 50. 51. 54. 55. 57.
 58. 60. 67. 69. 72. 80. 83. 85. 87. 99.]
The number of unique values for feature NUMBER_HEATED_ROOMS: 33 - [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 25. 30. 31. 36. 57. 67. 72. 80. 83.]
The number of unique values for feature MULTI_GLAZE_PROPORTION: 101
The number of unique values for feature WINDOWS_DESCRIPTION: 5 

In [61]:
#Making categorical variables into numerical representeations:

gla_data3 = pd.get_dummies(gla_data2, columns = ['property_type', 'new_build', 'duration', 'category',     
       'CURRENT_ENERGY_RATING', 'Fare_Zone', 'PROPERTY_TYPE', 'BUILT_FORM'])
       
gla_data3.head()

,TOTAL_FLOOR_AREA,NUMBER_HABITABLE_ROOMS,NUMBER_HEATED_ROOMS,MULTI_GLAZE_PROPORTION,WINDOWS_DESCRIPTION,price_paid,Deprivation_Index,driving_distance_miles,Median(£)-2011,property_type_1.0,...,PROPERTY_TYPE_2.0,PROPERTY_TYPE_3.0,PROPERTY_TYPE_4.0,PROPERTY_TYPE_5.0,BUILT_FORM_0.0,BUILT_FORM_1.0,BUILT_FORM_2.0,BUILT_FORM_3.0,BUILT_FORM_4.0,BUILT_FORM_5.0
2,66.00,3.0,3.0,100.0,100.0,337000.0,3.240188,13.92,210000.0,0,...,1,0,0,0,0,0,0,0,1,0
3,137.00,7.0,7.0,100.0,100.0,657000.0,-2.300779,17.67,230000.0,1,...,0,0,0,1,0,0,0,0,0,1
4,99.07,4.0,4.0,100.0,100.0,570000.0,-2.432610,16.01,374950.0,1,...,0,0,0,1,0,0,0,0,1,0
5,99.07,4.0,4.0,100.0,100.0,770000.0,-2.432610,16.01,374950.0,1,...,0,0,0,1,0,0,0,0,1,0
6,127.00,6.0,6.0,100.0,100.0,745000.0,-0.286001,13.22,535000.0,0,...,0,0,0,1,0,0,0,0,1,0


In [62]:
#Scale Columns, converting raw numbers in scale from zero to one.

scale_vars = ['TOTAL_FLOOR_AREA', 'NUMBER_HABITABLE_ROOMS',
       'NUMBER_HEATED_ROOMS', 'MULTI_GLAZE_PROPORTION', 'WINDOWS_DESCRIPTION',
       'Deprivation_Index', 'driving_distance_miles']

scaler = MinMaxScaler()   
gla_data3[scale_vars] = scaler.fit_transform(gla_data3[scale_vars])
gla_data3.head()

,TOTAL_FLOOR_AREA,NUMBER_HABITABLE_ROOMS,NUMBER_HEATED_ROOMS,MULTI_GLAZE_PROPORTION,WINDOWS_DESCRIPTION,price_paid,Deprivation_Index,driving_distance_miles,Median(£)-2011,property_type_1.0,...,PROPERTY_TYPE_2.0,PROPERTY_TYPE_3.0,PROPERTY_TYPE_4.0,PROPERTY_TYPE_5.0,BUILT_FORM_0.0,BUILT_FORM_1.0,BUILT_FORM_2.0,BUILT_FORM_3.0,BUILT_FORM_4.0,BUILT_FORM_5.0
2,0.006046,0.030303,0.036145,1.0,1.0,337000.0,0.521189,0.325843,210000.0,0,...,1,0,0,0,0,0,0,0,1,0
3,0.012611,0.070707,0.084337,1.0,1.0,657000.0,0.184577,0.413624,230000.0,1,...,0,0,0,1,0,0,0,0,0,1
4,0.009103,0.040404,0.048193,1.0,1.0,570000.0,0.176568,0.374766,374950.0,1,...,0,0,0,1,0,0,0,0,1,0
5,0.009103,0.040404,0.048193,1.0,1.0,770000.0,0.176568,0.374766,374950.0,1,...,0,0,0,1,0,0,0,0,1,0
6,0.011686,0.060606,0.072289,1.0,1.0,745000.0,0.306974,0.309457,535000.0,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
#Clear memory before running:
gla_data.info(verbose=False, memory_usage='deep')
gla_data2.info(verbose=False, memory_usage='deep')
gla_data3.info(verbose=False, memory_usage='deep')
del gla_data, gla_data2#, gla_plot

In [99]:
#Parse dataset class
class load_data(Dataset):
    #Load dataframe
    def __init__(self, dataframe):
        #Load pandas dataframe
        df = dataframe
        #Inputs and outputs as X and y
        self.X = df.drop('price_paid', axis=1).values.astype('float32')
        self.y = df['price_paid'].values.astype('float32')
        # Reshape target value
        self.y = self.y.reshape((len(self.y), 1))
        
    #Number of rows of dataframe
    def __len__(self):
        return len(self.X)
    
    #Get row of specific Index
    def __getitem__(self, i):
        return[self.X[i], self.y[i]]
    
    #Get indexes for train and test rows
    def get_splits(self, l_test=0.2):
        #determine size of test and train sets
        test_size = round(l_test * len(self.X))
        train_size = len(self.X) - test_size
        #Calculate split
        return random_split(self, [train_size, test_size])

In [100]:
#Before@ Thee hidden layers, 10 -> 8 -> 1

#model definition
class MLP(Module):
    #Define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        #Input from first to last hidden layers
        self.hidden1 = Linear(n_inputs, 256)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = ReLU()
#         self.act1 = Sigmoid()
        
        self.hidden2 = Linear(256, 128)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = ReLU()
#         self.act2 = Sigmoid()
        
        self.hidden3 = Linear(128, 64)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = ReLU()
        #self.act3 = Sigmoid()
        
        self.hidden4 = Linear(64, 32)
        xavier_uniform_(self.hidden4.weight)
        self.act4 = ReLU()
        #self.act4 = Sigmoid()
        
        self.hidden5 = Linear(32, 8)
        xavier_uniform_(self.hidden5.weight)
        self.act5 = ReLU()
#         self.act5 = Sigmoid()
        
        self.hidden6 = Linear(8, 1)
        xavier_uniform_(self.hidden6.weight)
        
    #Froward propagation input
    def forward(self, X):
        #Input first hidden layer to last layer
        X = self.hidden1(X)
        X = self.act1(X)
        
        X = self.hidden2(X)
        X = self.act2(X)
        
        X = self.hidden3(X)
        X = self.act3(X)
        
        X = self.hidden4(X)
        X = self.act4(X)
        
        X = self.hidden5(X)
        X = self.act5(X)
        
        X = self.hidden6(X)
        return X

In [101]:
#Prepare dataset:
def prepare_data(dataframe):
    dataset = load_data(dataframe)
    #Split the data
    train, test = dataset.get_splits()
    #Prepare DataLoader torch.utils
    train_DL = DataLoader(train, batch_size=32, shuffle=True)
    test_DL = DataLoader(test, batch_size=1024, shuffle=False)
    return train_DL, test_DL

In [108]:
#Train the model
def train_model(train_DL, model):
    #model tuning
    loss_function = MSELoss()
#     optimiser = SGD(model.parameters(), lr=0.001, momentum=0.9)
    optimiser = Adam(model.parameters(), lr=0.001)
    #Epochs
    for epoch in range(100):
        # Iterate over the DataLoader for training data
        for i, (inputs, targets) in enumerate(train_DL):
            #clear the gradients
            optimiser.zero_grad()
            #compute model output
            outputs = model(inputs)
            #calculte loss
            loss = loss_function(outputs, targets)
            #perform backward pass:
            loss.backward()
            #perform optimisation
            optimiser.step()
            print('Epoch:', epoch)
            

In [109]:
#Evaluate the model
def evaluate_model(test_DL, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_DL):
        #Evaluate model on test dataframe
        outputs = model(inputs)
        #Retrieve Numpy array:
        outputs = outputs.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        #store
        predictions.append(outputs)
        actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    #Calculate error (MSE)
    mse = mean_squared_error(actuals, predictions)
    return mse

In [110]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    output = model(row)
    # retrieve numpy array
    output = output.detach().numpy()
    return output

In [ ]:
#Execute model

#Test with a sample of data
gla_data4 = gla_data3.sample(frac=0.01)

#Prepare the data
train_DL, test_DL = prepare_data(gla_data4)
print(len(train_DL.dataset), len(test_DL.dataset))

#Define the neural network model
model = MLP(40)

#Train the model
train_model(train_DL, model)
print('Training ok')

#Evaluate the model
mse = evaluate_model(test_DL, model)
print('MSE: %.3f, RMSE %.3f' % (mse, np.sqrt(mse)))